In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from ema_workbench import (
    ScalarOutcome,
    load_results,
    Policy,
    MultiprocessingEvaluator
)
from ema_workbench.analysis import parcoords
from ema_workbench.util import ema_logging, utilities

from custom_problem_formulation_no_RfR import get_model_for_problem_formulation
from dike_model_function import DikeNetwork

In [3]:
# Load policies from the CSV file
policies_df = pd.read_csv('results/50_diverse_policies.csv')

# Drop unwanted columns
policies_df = policies_df.drop(columns=[
    'A.1_External Costs',
    'A.1_Expected Number of Deaths',
    'A.1_Expected Annual Damage',
    'A.2_External Costs',
    'A.2_Expected Number of Deaths',
    'A.2_Expected Annual Damage',
    'composite_ooi',
    'satisfied'
])

# Create a new column with the desired format
policies_df['index'] = 'scenario' + policies_df['scenario'].astype(str) + '_policy' + policies_df['Unnamed: 0'].astype(str)

# Set the new column as the index
policies_df.set_index('index', inplace=True)

# Drop the original 'scenario' and 'unnamed: 0' columns
policies_df = policies_df.drop(columns=['scenario', 'Unnamed: 0'])

policies_df.head()


,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.2_DikeIncrease 2,A.3_DikeIncrease 0,A.3_DikeIncrease 1,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2
index,,,,,,,,,,,,,,,,,,,,,
scenario48226_policy61,0,0,0,0,1,1,1,1,1,1,...,0,5,5,1,7,0,4,0,6,7
scenario5666_policy117,0,0,1,0,0,0,1,1,1,1,...,0,4,4,0,5,0,4,5,0,0
scenario5666_policy138,0,0,0,1,0,0,1,1,1,1,...,0,5,7,7,6,2,7,4,0,5
scenario5666_policy143,1,0,0,0,0,0,1,1,1,1,...,0,5,5,6,4,5,8,7,0,3
scenario5666_policy132,0,0,0,0,1,0,1,1,1,1,...,0,1,2,5,6,9,0,3,7,1


In [4]:
policies = []
for idx, row in policies_df.iterrows():
    policy_dict = row.to_dict()
    policy_name = idx
    policies.append(Policy(policy_name, **policy_dict))

model, steps = get_model_for_problem_formulation()

# Define the number of scenarios
n_scenarios = 1000

# Perform experiments
with MultiprocessingEvaluator(model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios,
                                            policies)

experiments, outcomes = results

/Users/wouter/.pyenv/versions/3.12.2/envs/EPA141A_Model_Based_Decision_Making_Env/lib/python3.12/site-packages/ema_workbench/em_framework/__init__.py:101: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn("ipyparallel not installed - IpyparalleEvaluator not available")
/Users/wouter/.pyenv/versions/3.12.2/envs/EPA141A_Model_Based_Decision_Making_Env/lib/python3.12/site-packages/ema_workbench/em_framework/__init__.py:101: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn("ipyparallel not installed - IpyparalleEvaluator not available")
/Users/wouter/.pyenv/versions/3.12.2/envs/EPA141A_Model_Based_Decision_Making_Env/lib/python3.12/site-packages/ema_workbench/em_framework/__init__.py:101: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn("ipyparallel not installed - IpyparalleEvaluator not available")
/Users/wouter/.pyenv/versions/3.12.2/envs/EPA141A_Model_Based_Decis

In [6]:
# Saving the results
utilities.save_results(results, 'results/50_policies_1000_scenarios.tar.gz')

In [26]:
outcomes = pd.DataFrame(outcomes)
experiments = pd.DataFrame(experiments)
policies_50_results = outcomes.join(experiments)

In [42]:
policies_50_results.sort_values('A.1_External Costs', ascending=False)

,A.1_External Costs,A.1_Expected Number of Deaths,A.1_Expected Annual Damage,A.2_External Costs,A.2_Expected Number of Deaths,A.2_Expected Annual Damage,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,...,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,scenario,policy,model
39998,1.083851e+08,0.000000,0.000000e+00,5.335857e+07,0.006620,7.513152e+07,121,43.388625,1.5,0.977557,...,5,9,5,7,2,3,4,998,scenario48226_policy50,dikesnet
14170,1.083851e+08,0.000922,1.410938e+07,5.941010e+07,0.001462,1.298660e+07,110,50.259451,10.0,0.366519,...,3,2,7,1,5,4,4,170,scenario5666_policy111,dikesnet
14157,1.083851e+08,0.002440,1.907082e+07,5.941010e+07,0.000000,0.000000e+00,8,349.515498,10.0,0.280368,...,3,2,7,1,5,4,4,157,scenario5666_policy111,dikesnet
14158,1.083851e+08,0.000000,0.000000e+00,5.941010e+07,0.000000,0.000000e+00,60,142.307477,10.0,0.495278,...,3,2,7,1,5,4,4,158,scenario5666_policy111,dikesnet
14159,1.083851e+08,0.000000,0.000000e+00,5.941010e+07,0.000000,0.000000e+00,7,348.677641,10.0,0.678904,...,3,2,7,1,5,4,4,159,scenario5666_policy111,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15325,0.000000e+00,0.000000,0.000000e+00,6.604510e+07,0.000000,0.000000e+00,70,203.073288,1.0,0.970736,...,2,3,6,5,1,1,4,325,scenario48226_policy2,dikesnet
15324,0.000000e+00,0.000000,0.000000e+00,6.604510e+07,0.000000,0.000000e+00,92,334.570171,1.5,0.381231,...,2,3,6,5,1,1,4,324,scenario48226_policy2,dikesnet
15323,0.000000e+00,0.000000,0.000000e+00,6.604510e+07,0.000000,0.000000e+00,104,228.739324,1.5,0.622888,...,2,3,6,5,1,1,4,323,scenario48226_policy2,dikesnet
15322,0.000000e+00,0.000000,0.000000e+00,6.604510e+07,0.000000,0.000000e+00,62,67.254598,1.5,0.533493,...,2,3,6,5,1,1,4,322,scenario48226_policy2,dikesnet


In [8]:
# Function to calculate SNR metric
def calculate_snr(policy, direction):
  mean_performance = np.mean(policy)
  std_dev = np.std(policy)
  
  if direction == "MINIMIZE":
      snr = mean_performance * std_dev
  else:
      snr = mean_performance / std_dev
  return snr


In [36]:
# Initialize an empty list to store all SNR scores
snr_scores = []

# Iterate over each policy in the experiments DataFrame
for policy in experiments['policy'].unique():
    # Initialize a dictionary to store SNR scores for the current policy
    scores = {'policy': policy}
    
    # Filter experiments DataFrame to get rows where 'policy' matches the current policy
    logical = experiments['policy'] == policy
    
    # Iterate over each outcome in the model.outcomes list (replace with your actual outcomes)
    for outcome in model.outcomes:
        # Extract values from the experiments DataFrame where the logical condition is True
        value = outcomes[outcome.name][logical]
        
        # Calculate the signal-to-noise ratio (SNR) for the extracted values
        sn_ratio = calculate_snr(value, outcome.kind)
        
        # Store the SNR score in the scores dictionary with outcome name as key
        scores[outcome.name] = sn_ratio
    
    # Append the scores dictionary to the snr_scores list
    snr_scores.append(scores)

# Convert the list of dictionaries to a DataFrame
snr_scores_df = pd.DataFrame(snr_scores)

# Set policy column as
snr_scores_df.set_index('policy', inplace=True)

/var/folders/mz/j0q1_7_x2f59p13bv_c81xhc0000gn/T/ipykernel_20948/1371807315.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  snr = mean_performance / std_dev
/var/folders/mz/j0q1_7_x2f59p13bv_c81xhc0000gn/T/ipykernel_20948/1371807315.py:9: RuntimeWarning: invalid value encountered in scalar divide
  snr = mean_performance / std_dev


In [37]:
snr_scores_df

,A.1_External Costs,A.1_Expected Number of Deaths,A.1_Expected Annual Damage,A.2_External Costs,A.2_Expected Number of Deaths,A.2_Expected Annual Damage
policy,,,,,,
scenario48226_policy61,8.926420e+15,0.197742,0.192009,inf,0.169889,0.171190
scenario5666_policy117,5.000309e+15,0.403515,0.393419,inf,0.171811,0.171231
scenario5666_policy138,inf,0.384558,0.376595,inf,0.161789,0.162065
scenario5666_policy143,4.779704e+15,0.305179,0.295224,inf,0.195899,0.193553
scenario5666_policy132,8.926420e+15,0.370034,0.363004,4.432212e+15,0.143188,0.145944
scenario5666_policy91,NaN,0.292446,0.288155,4.432212e+15,0.142286,0.145456
scenario24626_policy72,inf,0.465191,0.455724,4.432212e+15,0.140298,0.142542
scenario5666_policy150,inf,0.256697,0.249414,inf,0.122314,0.125551
scenario48226_policy3,inf,0.266501,0.261999,4.432212e+15,0.143677,0.146941


In [47]:
filtered_scores = []

for scores in snr_scores:
    inf_present = False
    nan_present = False
    
    # Check each value in scores dictionary for inf and nan
    for value in scores.values():
        if isinstance(value, (int, float)):
            if np.isinf(value):
                inf_present = True
            if np.isnan(value):
                nan_present = True
    
    # Append scores if no inf values and at least one non-NaN value
    if inf_present or nan_present:
        filtered_scores.append(scores)

# Convert filtered scores to DataFrame
filtered_scores_df = pd.DataFrame(filtered_scores)


In [48]:
filtered_scores_df

,policy,A.1_External Costs,A.1_Expected Number of Deaths,A.1_Expected Annual Damage,A.2_External Costs,A.2_Expected Number of Deaths,A.2_Expected Annual Damage
0,scenario48226_policy61,8.926420e+15,0.197742,0.192009,inf,0.169889,0.171190
1,scenario5666_policy117,5.000309e+15,0.403515,0.393419,inf,0.171811,0.171231
2,scenario5666_policy138,inf,0.384558,0.376595,inf,0.161789,0.162065
3,scenario5666_policy143,4.779704e+15,0.305179,0.295224,inf,0.195899,0.193553
4,scenario5666_policy91,NaN,0.292446,0.288155,4.432212e+15,0.142286,0.145456
5,scenario24626_policy72,inf,0.465191,0.455724,4.432212e+15,0.140298,0.142542
6,scenario5666_policy150,inf,0.256697,0.249414,inf,0.122314,0.125551
7,scenario48226_policy3,inf,0.266501,0.261999,4.432212e+15,0.143677,0.146941
8,scenario24626_policy74,inf,0.425644,0.415955,4.432212e+15,0.143143,0.144530
9,scenario5666_policy98,8.926420e+15,0.370034,0.363004,inf,0.190363,0.187723


In [49]:

# Initialize an empty list to store all SNR scores
snr_scores = []

# Iterate over each policy in the experiments DataFrame
for policy in experiments['policy'].unique():
    print(f"Processing policy: {policy}")
    
    # Initialize a dictionary to store SNR scores for the current policy
    scores = {'policy': policy}
    
    # Filter experiments DataFrame to get rows where 'policy' matches the current policy
    logical = experiments['policy'] == policy
    
    # Iterate over each outcome in the model.outcomes list (replace with your actual outcomes)
    for outcome in model.outcomes:
        print(f"Calculating SNR for outcome: {outcome.name}")
        
        # Extract values from the outcomes DataFrame where the logical condition is True
        value = outcomes[outcome.name][logical]
        
        # Print information about the extracted values for debugging
        print(f"Values extracted for {outcome.name}:")
        print(value)
        
        # Calculate the signal-to-noise ratio (SNR) for the extracted values
        sn_ratio = calculate_snr(value, outcome.kind)
        
        # Print SNR value calculated for debugging
        print(f"SNR calculated for {outcome.name}: {sn_ratio}")
        
        # Store the SNR score in the scores dictionary with outcome name as key
        scores[outcome.name] = sn_ratio
    
    # Append the scores dictionary to the snr_scores list
    snr_scores.append(scores)

# Convert the list of dictionaries to a DataFrame
snr_scores_df = pd.DataFrame(snr_scores)

# Set policy column as index
snr_scores_df.set_index('policy', inplace=True)

# Display or further process the DataFrame as needed
print(snr_scores_df)


Processing policy: scenario48226_policy61
Calculating SNR for outcome: A.1_External Costs
Values extracted for A.1_External Costs:
0      6.650701e+07
1      6.650701e+07
2      6.650701e+07
3      6.650701e+07
4      6.650701e+07
           ...     
995    6.650701e+07
996    6.650701e+07
997    6.650701e+07
998    6.650701e+07
999    6.650701e+07
Name: A.1_External Costs, Length: 1000, dtype: float64
SNR calculated for A.1_External Costs: 8926420440478654.0
Calculating SNR for outcome: A.1_Expected Number of Deaths
Values extracted for A.1_Expected Number of Deaths:
0      0.000000
1      0.000000
2      0.000000
3      0.000000
4      0.000000
         ...   
995    0.000000
996    0.000000
997    0.164613
998    0.000000
999    0.000000
Name: A.1_Expected Number of Deaths, Length: 1000, dtype: float64
SNR calculated for A.1_Expected Number of Deaths: 0.19774169005442976
Calculating SNR for outcome: A.1_Expected Annual Damage
Values extracted for A.1_Expected Annual Damage:
0      0

/var/folders/mz/j0q1_7_x2f59p13bv_c81xhc0000gn/T/ipykernel_20948/1371807315.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  snr = mean_performance / std_dev
/var/folders/mz/j0q1_7_x2f59p13bv_c81xhc0000gn/T/ipykernel_20948/1371807315.py:9: RuntimeWarning: invalid value encountered in scalar divide
  snr = mean_performance / std_dev


In [56]:
outcomes['A.1_External Costs'].value_counts()

A.1_External Costs
3.269490e+07    15000
6.650701e+07    13000
0.000000e+00     7000
3.725520e+07     3000
7.122315e+07     3000
7.709999e+07     3000
1.014745e+08     3000
1.083851e+08     2000
7.222271e+07     1000
Name: count, dtype: int64